In [1]:
!pip install git+https://github.com/UriKH/RamanujansDreams.git

  Cloning https://github.com/UriKH/RamanujansDreams.git to c:\users\urikh\appdata\local\temp\pip-req-build-0tgk0n1q
  Resolved https://github.com/UriKH/RamanujansDreams.git to commit 84e8be8c8fbefb894af78bd09c4e39c4186e1960
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/RamanujanMachine/ramanujantools.git to c:\users\urikh\appdata\local\temp\pip-install-0n8wsajv\ramanujantools_be0bf5054ded435b8959fd002b661a65
  Resolved https://github.com/RamanujanMachine/ramanujantools.git to commit b897a1cf0c1aba0ec50171aae412be5bbded61b0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting

  Running command git clone --filter=blob:none --quiet https://github.com/UriKH/RamanujansDreams.git 'C:\Users\urikh\AppData\Local\Temp\pip-req-build-0tgk0n1q'
  hint: core.useBuiltinFSMonitor=true is deprecated;please set core.fsmonitor=true instead
  hint: Disable this message with "git config set advice.useCoreFSMonitorConfig false"
  Running command git clone --filter=blob:none --quiet https://github.com/RamanujanMachine/ramanujantools.git 'C:\Users\urikh\AppData\Local\Temp\pip-install-0n8wsajv\ramanujantools_be0bf5054ded435b8959fd002b661a65'
  hint: core.useBuiltinFSMonitor=true is deprecated;please set core.fsmonitor=true instead
  hint: Disable this message with "git config set advice.useCoreFSMonitorConfig false"
  Running command git clone --filter=blob:none --quiet https://github.com/RamanujanMachine/LIReC.git 'C:\Users\urikh\AppData\Local\Temp\pip-install-0n8wsajv\lirec_5a70a549d90348bc8fc2e92a8fdca8f0'
  hint: core.useBuiltinFSMonitor=true is deprecated;please set core.fsmo

## Imports and configuration
From `dreamer` are available the sytem itself, its configurations and the different stages.

In [2]:
from dreamer import System, config
from dreamer import analysis, search, extraction
from dreamer.loading import *

import sympy as sp # import for fraction usage

Default behavior is changable via the `config.configure()` and the usage of the different stages (configurations are grouped by stage).

In [3]:
trajectory_compute_func = (lambda d: max(10 ** (d-1) / 2, 10))

config.configure(
    system={
        'EXPORT_CMFS': './mycmfs',                          # export CMF as objects to directory: ./mycmfs
        'EXPORT_ANALYSIS_PRIORITIES': './myshards',         # export shards found in analysis into: ./myshards
        'EXPORT_SEARCH_RESULTS': './mysearchresults'        # export the search results into: ./mysearchresults
    },
    analysis={
        'IDENTIFY_THRESHOLD': 0.2,          # ignore shards with less than 20% identified trajectories as converge
                                            # to the constant
        'NUM_TRAJECTORIES_FROM_DIM': trajectory_compute_func  # number of trajectories to be auto generated in analysis
    },
    search={
        'NUM_TRAJECTORIES_FROM_DIM': trajectory_compute_func  # number of trajectories to be auto generated in search if needed by the module
    }
)


You can also view the configurations like so:

In [4]:
# view system configs
print(config.system.display())

+----------------------------+--------------------------------------------------------------+
| Property                   | Value                                                        |
+----------------------------+--------------------------------------------------------------+
| CONSTANTS                  | []                                                           |
| MODULE_ERROR_SHOW_TRACE    | True                                                         |
| TQDM_CONFIG                | {'bar_format': '{desc:<40}     {bar} | {elapsed} {rate_fm... |
| LOGGING_BUFFER_SIZE        | 150                                                          |
| USE_LIReC                  | True                                                         |
| EXPORT_CMFS                | ./mycmfs                                                     |
| EXPORT_ANALYSIS_PRIORITIES | ./myshards                                                   |
| EXPORT_SEARCH_RESULTS      | ./mysearchresults            

In [5]:
# view analysis configs
print(config.analysis.display())

+----------------------------+-------------------------------------------+
| Property                   | Value                                     |
+----------------------------+-------------------------------------------+
| USE_CACHING                | True                                      |
| NUM_TRAJECTORIES_FROM_DIM  | <function <lambda> at 0x0000022798895440> |
| IDENTIFY_THRESHOLD         | 0.2                                       |
| PRINT_FOR_EVERY_SEARCHABLE | True                                      |
| SHOW_START_POINT           | True                                      |
| SHOW_SEARCHABLE            | False                                     |
| USE_LIReC                  | True                                      |
| ANALYZE_LIMIT              | False                                     |
| ANALYZE_EIGEN_VALUES       | False                                     |
| ANALYZE_GCD_SLOPE          | False                                     |
+------------------------

### System execution
Run the system on a ${}_2F_1$ with $z=\frac{1}{2}$ ($\pi$'s 3D simple CMF) with the basic shard analyzer and searcher (all trajectory) modules.

Note that in order to use the ${}_pF_q$ we need to tell for which constant we search inside the CMF and give a shift type of start points - i.e. all start points in this case are of the form $(x, y, z+\frac{1}{2})$ where $x,y,z\in\mathbb{Z}$.

#### Constants:
- To import known constants use: `from dreamer import pi, e, euler_gamma, catalan`
- To import known functions:
  ```
  from dreamer import zeta, log, sqrt, power

  log_2 = log(2)
  zeta_3 = zeta(3)
  sqrt_2 = sqrt(2)
  pi_p_2 = power(pi, 2)
  ```
- If constant is not provided by the system you can add it yourself by simply:
  ```
  from dreamer import Constant

  my_cons = Constant('const-name', <value using sympy>)
  ```

In [6]:
from dreamer import pi, zeta
import sys

sys.setrecursionlimit(5000)

System(
    if_srcs=[pFq_formatter(pi, 2, 1, sp.Rational(1, 2), [0, 0, sp.Rational(1, 2)])],
    extractor=extraction.extractor.ShardExtractorMod,
    analyzers=[analysis.AnalyzerModV1],
    searcher=search.SearcherModV1
).run(constants=[pi])


[INFO] Loading CMFs ...

[INFO] CMFs for pi exported to ./mycmfs\pi

[INFO] Searching for pi using inspiration functions: 
1. ShiftCMF(cmf=pFq((2, 1, 1/2)), shift=Position({x0: 0, x1: 0, y0: 1/2}), raw=False)



Extracting shards for "pi"                                                     | 00:00 ?it/s (0.00%)
Computing shards                                                               | 00:00 ?it/s (0.00%)

Computing shard encodings                                                      | 00:00 ?it/s (0.00%)

                                                                                                    
Extracting shards for "pi"                                                     | 00:00 ?it/s (0.00%)
Computing shards                                                               | 00:00 ?it/s (0.00%)

[INFO] In CMF no. 1: found 2 hyperplanes and 4 shards 




Creating shard objects                                                         | 00:00 ?it/s (0.00%)

                                                                                                    
Computing shards                             ███████████████████████████ | 00:00  2.10it/s (100.00%)
Extracting shards for "pi"                   ███████████████████████████ | 00:00  2.05it/s (100.00%)
Analyzing constants and their CMFs                                             | 00:00 ?it/s (0.00%)


================================================================== Analyzing for pi ==================================================================


...

                                                                                                    
Analyzing constants and their CMFs                                             | 00:20 ?it/s (0.00%)
                                                                                                    
Analyzing constants and their CMFs                                             | 00:20 ?it/s (0.00%)
Analyzing shards                             ██████████████               | 00:20  7.20s/it (50.00%)
                                                                                                    
Analyzing constants and their CMFs                                             | 00:20 ?it/s (0.00%)
                                                                                                    
Analyzing constants and their CMFs                                             | 00:20 ?it/s (0.00%)
Analyzing shards                             █████████████████████        | 00:20  6.61s/it

[INFO] Identified 0.00% of trajectories as containing "pi"
[WARNING] No best delta was found

========== SHARD NO. 4 ==========
[INFO] Chosen shard start point: Position({x0: 2, x1: 2, y0: 5/2})


                                                                                                    
Analyzing constants and their CMFs                                             | 00:28 ?it/s (0.00%)
Analyzing shards                             █████████████████████        | 00:28  6.61s/it (75.00%)
Analyzing shards                             ███████████████████████████ | 00:28  7.04s/it (100.00%)


[INFO] Identified 100.00% of trajectories as containing "pi"
    > Best delta:	 -0.1532
    > Trajectory:	 SearchVector(start=Position({x0: 2, x1: 2, y0: 5/2}), trajectory=Position({x0: 1, x1: 2, y0: 0}))
    > CMF:		 pFq((2, 1, 1/2))
    > p,q vectors:	[[0, 2], [1, 0]]

[INFO] Priorities for pi exported to ./myshards\pi



[INFO] Best delta for "pi" found by the searcher is -0.15315491774890688
* Trajectory: Position({x0: 2, x1: 1, y0: 0}) 
* Start: Position({x0: 2, x1: 2, y0: 5/2})
